# Main

### Prerequisites

**Imports**

In [ ]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

**Fix directories, define path variables**

In [ ]:
# make sure notebook is ran from src
cwd = os.getcwd()
if not cwd.split('/')[-1] == 'src':
    try:
        os.chdir('src')
    except FileNotFoundError:
        print('Error: please run from src dir or project root')
        sys.exit(1)

# define paths
data_path = '../data/'
csv_path = data_path + 'csv/'
shp_path = data_path + 'shapefiles/'
plot_path = '../plots/'
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

---

### Read data

In [ ]:
# these are just manhattan gdfs, with correct projection
gdf_streets: gpd.GeoDataFrame = gpd.read_file(shp_path + 'streets/M_streets.shp')
gdf_traffic: gpd.GeoDataFrame = gpd.read_file(shp_path + 'traffic/M_traffic.shp')
gdf_trees: gpd.GeoDataFrame = gpd.read_file(shp_path + 'trees/M_trees.shp')

Now we create a grid and assign the relevant data to each cell.

In [ ]:
# define a bounding box around Manhattan
xmin, ymin, xmax, ymax = gdf_streets.total_bounds
bbox = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)])

# define granularity of grid
l, w = .005, .005

# create a grid of polygons
cols = list(np.arange(xmin, xmax+w, w))
rows = list(np.arange(ymin, ymax+l, l))
polygons = []
for x in cols[:-1]:
    for y in rows[:-1]:
        polygons.append(Polygon([(x, y), (x, y+l), (x+w, y+l), (x+w, y), (x, y)]))

# create a gdf from the grid
grid = gpd.GeoDataFrame({'geometry': polygons})
grid.crs = 4326

In [ ]:
# plot the grid
fig, ax = plt.subplots(figsize=(10, 10))
gdf_streets.plot(ax=ax, color='grey', alpha=0.5, zorder=1)
grid.plot(ax=ax, color='none', edgecolor='black', linewidth=1, zorder=2)
gdf_trees.plot(ax=ax, markersize=0.05, color='green', alpha=0.1, zorder=3)
gdf_traffic.plot(ax=ax, column='Avg_Vol', markersize=5, cmap='Reds', zorder=3)
ax.set_axis_off()